In [61]:
# -*- coding: utf-8 -*-
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu

In [62]:
# Read recipe inputs
y1985_MA = dataiku.Dataset("y1985_MA")
df = y1985_MA.get_dataframe()

## Functions ##

In [63]:
def position_indicator(col1, col2):
    '''Position indicator relative to another (E)MA
    0 : below ; 1 : above'''

    # Returns a boolean series
    s_comparison = col1 > col2

    # convert true/false to 0/1
    return s_comparison.astype(int)

In [64]:
def get_slope(ma_col):
    '''price difference / nb days (3)
    Normalized to initial starting point observation'''

    slope = ma_col - ma_col.shift(3)
    normalized_slope = round((slope/ma_col.shift(3)*100),1)

    return normalized_slope

In [65]:
def compute_distance(col1,col2):
    '''Compute relative distance between Close / (E)MA or between (E)MA
    Relative to MA or slower MA.
    Must be passed as 1st parameter'''

    return round( ((col2 - col1)/col1)*100 ,2)

In [96]:
def stdev_breakout(col, w):
    '''Measures if last price variation is above stdev 10, 20 days
    1 if breakout observed'''
    
    s_std = col.rolling(w).std()
    s_price_var = abs(col - col.shift(1))
    
    s_breakout = s_price_var > s_std
    return s_breakout.astype(int)

## Study 1 : 3 MA relative positions ##
4-9-18 and 9-20-200

In [66]:
df["position_MA4_over_MA9"] = position_indicator(df.MA4, df.MA9)
df["position_MA4_over_MA18"] = position_indicator(df.MA4, df.MA18)
df["position_MA9_over_MA18"] = position_indicator(df.MA9, df.MA18)

df["position_MA9_over_MA20"] = position_indicator(df.MA9, df.MA20)
df["position_MA9_over_MA200"] = position_indicator(df.MA9, df.MA200)
df["position_MA20_over_MA200"] = position_indicator(df.MA20, df.MA200)

## Study 2 : O'Reilly Python for Finance ##

In [67]:
# Best combination after testing in book
df["OReilly_MA40_over_MA149"] = position_indicator(df.MA40, df.MA149)

## Study 3 : Elder's EMA ##

In [68]:
df["Elder_EMA13_EMA26"] = position_indicator(df.EMA13, df.EMA26)

## Study 4 : Oliver Kell ##
Financial Wisdom, (part of) Best trading strategy 2021

In [69]:
df["Kell_EMA10_over_EMA20"] = position_indicator(df.EMA10, df.EMA20)
df["Kell_EMA10_over_EMA30"] = position_indicator(df.EMA10, df.EMA30)
df["Kell_EMA20_over_EMA30"] = position_indicator(df.EMA20, df.EMA30)

## Study 5 : (E)MA slopes ##

In [70]:
df["MA4_slope"] = get_slope(df.MA4)
df["MA9_slope"] = get_slope(df.MA9)
df["EMA20_slope"] = get_slope(df.EMA20)
df["EMA30_slope"] = get_slope(df.EMA30)

In [71]:
# Sum of slopes
df["Slopes_sum"] = round(df["MA4_slope"] + df["MA9_slope"] + df["EMA20_slope"]+ df["EMA30_slope"] ,2)
df["Slopes_avg"] = round(df["Slopes_sum"]/4, 2)

In [72]:
df = df.loc[3:]

## Study 6 : Relative Price Position to (E)MAs ##

In [73]:
df["dist_Close_MA9"] = compute_distance(df.MA9,df.Close)
df["dist_Close_MA18"] = compute_distance(df.MA18,df.Close)
df["dist_Close_MA40"] = compute_distance(df.MA40,df.Close)

## Study 7 : Relative MA distance ##

In [74]:
df["dist_EMA13_EMA26"] = compute_distance(df.EMA26,df.EMA13)
df["dist_EMA20_EMA30"] = compute_distance(df.EMA30,df.EMA20)

df["dist_MA4_MA18"] = compute_distance(df.MA18,df.MA4)
df["dist_MA9_MA40"] = compute_distance(df.MA40,df.MA9)

## Study 8 : EMA StDev breakout ##

In [101]:
# Adding rolling sum to indicate that breakout occured at least once, max 5 days ago
df["StDev_breakout_10"] = stdev_breakout(df.Close, 10).rolling(5).sum()
df["StDev_breakout_20"] = stdev_breakout(df.Close, 20).rolling(5).sum()

In [104]:
df = df.loc[7:]
df.reset_index(drop=True, inplace=True)

## End, write results to Flow ##

In [0]:
# Write recipe outputs
y1985_MA_enriched = dataiku.Dataset("y1985_MA_enriched")
y1985_MA_enriched.write_with_schema(df)